In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
from  torchvision import datasets

cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True )
data = cifar_trainset.data / 255

cifar_10_mean = data.mean(axis=(0, 1, 2)).tolist()
cifar_10_std = data.std(axis=(0, 1, 2)).tolist()
print(f"Mean : {cifar_10_mean}, Standard deviation: {cifar_10_std}")

100%|██████████| 170M/170M [00:04<00:00, 42.2MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Mean : [0.4913996786152028, 0.4821584083946074, 0.4465309144454644], Standard deviation: [0.24703223246328238, 0.2434851280000556, 0.26158784172796423]


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 100
batch_size = 128
learning_rate = 0.001

In [ ]:
# Albumentation transforms
transform_train = A.Compose([
    A.HorizontalFlip(),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.CoarseDropout(max_holes=1, max_height=16, max_width=16, min_holes=1, min_height=16, min_width=16,
                    fill_value=cifar_10_mean, mask_fill_value=None, p=0.5),
    A.Normalize(mean=cifar_10_mean, std=cifar_10_std),
    ToTensorV2(),
])


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar_10_mean, std=cifar_10_std),
])

In [ ]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=lambda img: transform_train(image=np.array(img))['image'])
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# has the architecture with convolutional blocks - C1 | C2 | C3 | C4 | Output
    # (No MaxPooling, but convolutions, where the last one layer in each block has a stride of 2 instead)
    # (No restriction on using 1x1)
    # (If you can figure out how to use Dilated kernels here instead of MP or strided convolution, then 200pts extra!)
# total Receptive Field must be more than 44
# one of the layers must use Depthwise Separable Convolution
# one of the layers must use Dilated Convolution
# use GAP (compulsory): add FC after GAP to target #of classes (optional)


# Convolutional neural network (CNN)
class CIFAR10_ConvNet(nn.Module):
    def __init__(self):
        super(CIFAR10_ConvNet, self).__init__()
        dropout_value = 0.1
        # Block - 1
        # Input: 3x32x32
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout(dropout_value),
            # ouptput: 32, RF: 3

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(dropout_value),
            # output: 32, RF: 5

            # Depthwise separable convolution
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), stride=1, padding=1, groups=32),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1, 1), stride=1, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # output: 32, RF: 7

            # Layer Dilated Convolution
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=2, padding=2, dilation=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(dropout_value),
            # output: 16, RF: 11

        )

        self.transition1 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=16, kernel_size=(1, 1), padding=0, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU()
            #output : 16, RF: 11
        )

        # Block - 2
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(dropout_value),
            # output: 16, RF: 15

            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(dropout_value),
            # output: 16, RF: 19

            # Layer Dilated Convolution
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=2, padding=2, dilation=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(dropout_value),
            # output: 8, RF: 27
        )

        self.transition2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=16, kernel_size=(1, 1), padding=0, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU()
            # output: 8, RF: 27
        )

        # Block - 3
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(dropout_value),
            # output: 8, RF: 35

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(dropout_value),
            # output: 8, RF: 43

            # Layer Dilated Convolution
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=2, padding=2, dilation=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(dropout_value),
            # output: 4, RF: 59
        )

        self.transition3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=16, kernel_size=(1, 1), padding=0, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU()
            # output: 4, RF: 59
        )

        # Block - 4
        self.convblock4 = nn.Sequential(
            # Layer Dilated Convolution
            nn.Conv2d(in_channels=16, out_channels=64, kernel_size=(3, 3), stride=2, padding=2, dilation=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # output: 2, RF: 91
        )

        # Output Block
        self.outputblock = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels=64, out_channels=10, kernel_size=(1, 1), padding=0, bias=False)
        )


    def forward(self, x):
        x = self.convblock1(x)
        x = self.transition1(x)
        x = self.convblock2(x)
        x = self.transition2(x)
        x = self.convblock3(x)
        x = self.transition3(x)
        x = self.convblock4(x)
        x = self.outputblock(x)
        x = x.view(-1, 10)
        return x

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = CIFAR10_ConvNet().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
           Dropout-4           [-1, 16, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           4,608
       BatchNorm2d-6           [-1, 32, 32, 32]              64
              ReLU-7           [-1, 32, 32, 32]               0
           Dropout-8           [-1, 32, 32, 32]               0
            Conv2d-9           [-1, 32, 32, 32]             320
           Conv2d-10           [-1, 64, 32, 32]           2,112
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 16, 16]          36,864
      BatchNorm2d-14           [-1

In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, criterion, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = criterion(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
model = CIFAR10_ConvNet().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# step_lr
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
                            optimizer,
                            max_lr=0.1,
                            steps_per_epoch=len(train_loader),
                            epochs=num_epochs
                        )

# train loop
for epoch in range(1, num_epochs + 1):
    print("Epoch: ", epoch)
    print("Learning rate: ", optimizer.state_dict()['param_groups'][0]['lr'])
    train(model, device, train_loader, criterion, optimizer, epoch)
    test(model, device, test_loader, criterion)
    lr_scheduler.step()

Epoch:  1
Learning rate:  0.0040000000000000036


Loss=1.674565315246582 Batch_id=390 Accuracy=33.39: 100%|██████████| 391/391 [00:49<00:00,  7.85it/s]



Test set: Average loss: 0.0120, Accuracy: 4389/10000 (43.89%)

Epoch:  2
Learning rate:  0.004000001721825006


Loss=1.3594435453414917 Batch_id=390 Accuracy=44.87: 100%|██████████| 391/391 [00:42<00:00,  9.26it/s]



Test set: Average loss: 0.0106, Accuracy: 5081/10000 (50.81%)

Epoch:  3
Learning rate:  0.004000006887299928


...


Test set: Average loss: 0.0032, Accuracy: 8621/10000 (86.21%)

Epoch:  99
Learning rate:  0.004016535458033821


Loss=0.4407823979854584 Batch_id=390 Accuracy=80.03: 100%|██████████| 391/391 [00:40<00:00,  9.60it/s]



Test set: Average loss: 0.0035, Accuracy: 8550/10000 (85.50%)

Epoch:  100
Learning rate:  0.004016874618212443


Loss=0.4359794557094574 Batch_id=390 Accuracy=79.97: 100%|██████████| 391/391 [00:40<00:00,  9.66it/s]



Test set: Average loss: 0.0033, Accuracy: 8594/10000 (85.94%)



In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt

def plot_training_curve(train_losses, train_acc, test_losses, test_acc):
    t = [t_items.item() for t_items in train_losses]
    fig, axs = plt.subplots(2,2,figsize=(15,10))
    axs[0, 0].plot(t)
    axs[0, 0].set_title("Training Loss")
    axs[1, 0].plot(train_acc[4000:])
    axs[1, 0].set_title("Training Accuracy")
    axs[0, 1].plot(test_losses)
    axs[0, 1].set_title("Test Loss")
    axs[1, 1].plot(test_acc)
    axs[1, 1].set_title("Test Accuracy")

In [ ]:
plot_training_curve(train_losses, train_acc, test_losses, test_acc)